In [1]:
import pandas as pd
import pickle as pkl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import numba as nb
import os
from pathlib import Path
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 80
from matplotlib import rcParams
rcParams['figure.figsize'] = 8,4
sns.set_context("paper")
sns.set(font='serif')
sns.set_style("white", {
        "font.family": "serif",
        "font.serif": ["Times", "Palatino", "serif"]})

pardir=Path(os.getcwd()).parent.parent
datadir=str(pardir) +'/Data'


############
###########
# this is how much to absorb into the boundary (redundant, I don't use that anymore)
mysnap=0

In [2]:
from numba import float64, float32, int32, int64
#this function constructs the kernel
@nb.jit(float64[:](int32),nopython=True)
def K(band):
    return np.array([np.power(1-np.power((band-x)/band,2),3) for x in range(1,band+1)]
                    +[np.power(1-np.power(x/band,2),3) for x in range(1,band)], dtype=np.float64)*35/32

@nb.jit(nopython=True)
def K0(band):
    return np.array([6+18*x/band+12*x*x/(band*band) for x in range(0,band)])

#this function counts the occurences of each observation and places them on a grid
# x is price(horizontal), y is quality(vertical)
@nb.jit(nopython=True)
def gcounts(xdata, ydata, grid):
    for i in range(0,len(xdata)):
        grid[ydata[i],xdata[i]]+=1

@nb.jit(nopython=True)
def phipsi(q,grid, band_ps, band_qs):
    phi=psi=0
    bot=max(0,q-band_qs)
    top=min(100*fc-1,q+band_qs)
    for j in range(bot,top):
        for i in range(0,2*b1-1):       
            phi+=k1[i]*kq[j-bot-1]*grid[j,i]
        for i in range(0,b0-1):
            psi+=k0[b0-i-1]*kq[j-bot-1]*grid[j,i]
    return (np.log(1/(M*M) + phi/(M*b1*band_qs))-np.log(max(1/(M*M),psi/(M*b0*band_qs))))/b1

@nb.jit(nopython=True)
def tau(p,q, band_ps, band_qs, A):
    d=phipsi(q,grid, band_ps, band_qs)
    return int(p+d*p*p+A*d*d*p*p*p)

@nb.jit(nopython=True)
def taus_fun(reflect, grid, band_ps, band_qs, A):
    for i in range(0,100*fc+1):
        for j in range(0,100*fc+1):
            reflect[i,100*fc+j]+=grid[i,j]
            t=int(tau(j,i, band_ps, band_qs, A))
            if t<100*fc:
                reflect[i,100*fc-t-1]+=grid[i,j]

@nb.jit(int32(float64[:,:],float64[:,:], int32, int32, int32, float64[:]),nopython=True)
def smooth(reflect, reflect2, band_ps, band_qs, fc, lambd):
    Kband_ps=K(band_ps)
    bound1=100*fc+1
    bound2=200*fc+1
    for j in range(0,bound1):
        myband_qs=int(lambd[j]*band_qs)
        Kband_qs=K(myband_qs)
        bot=max(0,j-myband_qs)
        top=min(j+myband_qs-1,100*fc+1)
        for i in range(0,bound2):
            area=0            
            left=max(i-band_ps,0)
            right=min(i+band_ps-1, 200*fc+1)
            for kj in range(bot,top):
                mykj=Kband_qs[kj-j+myband_qs]
                for ki in range(left,right):
                    myki=Kband_ps[ki-i+band_ps]
                    my=myki*mykj
                    reflect2[j,i]+=reflect[kj,ki]*my
                    area+=my
            #reflect2[j,i]=reflect2[j,i]/area
    return 0

### At first I wanted to run the 2d kernel with fc=8, that is 800x800 matrix, but later I realized that it does not really change anything because I'm going to interpolate the non-integer points when solving the ODE in the next steps. So fc=4, that is 400x400, should be enough and also runs in a reasonable time.

In [3]:
for fc in [8]:
    print('fc=',fc)
    for industry in ["leg", "mil", "sci", "edu", "tec"]:
        for weight in ['55','80']:
            print('...'*25)
            print(industry, weight, '...')

            mydir=datadir+"/derive/"+industry+weight+"/"

            try:
                os.stat(mydir)
            except:
                os.mkdir(mydir) 

            sigma, haz=pkl.load(open(mydir+industry+weight+"j_strategy3_"+str(fc)+".pkl", "rb"))
            [PART,DATA]=pkl.load(open(datadir+"/derive/"+industry+weight+"j.pkl", "rb"))


            mydata31=DATA.loc[DATA.sapp_ps>mysnap].copy()
            mydata30=DATA.loc[DATA.sapp_ps<=mysnap].copy()
            w=int(weight)

            xdata=(mydata31.sapp_ps*fc).values
            ydata=(mydata31.sapp_qs*fc).values
            M=len(ydata)

            band_qs=int(1.06*2.987*np.power(M,-1/6)*np.std(ydata))
            band_ps=int(1.06*2.987*np.power(M,-1/6)*np.std(xdata))
            A=0.55
            b1=int(band_ps*np.power(M,-1/20))
            b0=int(1.48*b1)

            lambd=np.ones(100*fc+1, float)

#             for i in range(0,bstar*fc):
#                 lambd[i]=1-(1-bstar*fc/band_qs)*(bstar*fc-i)*(bstar*fc-i)/((bstar*fc)*(bstar*fc))
#             for i in range(0,bstar*fc):
#                 lambd[100*fc-i]=1-(1-bstar*fc/band_qs)*(bstar*fc-i)*(bstar*fc-i)/((bstar*fc)*(bstar*fc))
                
                
            #band_ps, band_qs, b0, b1

            ydata=ydata.astype(int)
            xdata=xdata.astype(int)

            grid=np.zeros((100*fc+1,100*fc+1), int)
            reflect=np.zeros((100*fc+1,200*fc+1), int)

            gcounts(xdata,ydata,grid)

            k1=K(b1)
            k0=K0(b0)
            kq=K(band_qs)

            reflect=np.zeros((100*fc+1,200*fc+1), dtype=np.float64)
            %time taus_fun(reflect, grid, band_ps, band_qs, A)

            reflect2=np.zeros((100*fc+1, 200*fc+1), dtype=np.float64)
            reflect.shape

            area=0
            reflect2=np.zeros((100*fc+1, 200*fc+1), dtype=np.float64)
            %time smooth(reflect,reflect2, band_ps, band_qs, fc, lambd)

            reflect3=reflect2[:,100*fc:]
            # to rescale back I need to multiply by fc*fc
            reflect3=fc*fc*reflect3/np.sum(reflect3)

            addr=mydir+industry+weight+"j_reflect3_"+str(fc)+".pkl"
            pkl.dump(reflect3,open(addr, "wb"))

fc= 8
...........................................................................
leg 55 ...
CPU times: user 4min 38s, sys: 16.7 ms, total: 4min 38s
Wall time: 4min 38s
CPU times: user 18min 59s, sys: 9.61 s, total: 19min 8s
Wall time: 19min 4s
...........................................................................
leg 80 ...
CPU times: user 3min 13s, sys: 103 ms, total: 3min 13s
Wall time: 3min 12s
CPU times: user 5min 12s, sys: 23.3 ms, total: 5min 12s
Wall time: 5min 11s
...........................................................................
mil 55 ...
CPU times: user 3min 44s, sys: 123 ms, total: 3min 44s
Wall time: 3min 44s
CPU times: user 8min 1s, sys: 33.3 ms, total: 8min 1s
Wall time: 7min 59s
...........................................................................
mil 80 ...
CPU times: user 3min 16s, sys: 2.38 s, total: 3min 18s
Wall time: 3min 17s
CPU times: user 5min 23s, sys: 3.97 s, total: 5min 27s
Wall time: 5min 27s
............................................